In [1]:
#!pip show torch
#!pip show torchtext

In [2]:
# Import necessary libraries
import pandas as pd


In [ ]:
df = pd.read_csv("data/movies_2015_2023_preprocessed_genre.csv")
df.head()

,title,release_year,language,genre,overview,vote_average,vote_count,popularity,cleaned_overview
0,Pad Man,2018,Indisch,Humor,upon realizing extent woman affected menses se...,7.420,200.0,7.036,upon realizing extent woman affected menses se...
1,Tamasha,2015,Indisch,Humor,meeting vacation ved tara sense connection vow...,6.720,141.0,8.770,meeting vacation ved tara sense connection vow...
2,Tu Jhoothi Main Makkaar,2023,Indisch,Humor,earn extra cash mickey help couple break life ...,6.253,144.0,10.045,earn extra cash mickey help couple break life ...
3,Hindi Medium,2017,Indisch,Humor,mita raj batra affluent couple delhi chandni c...,7.300,166.0,7.001,mita raj batra affluent couple delhi chandni c...
4,Dilwale,2015,Indisch,Humor,raj mafia member one day meet girl meera chasi...,6.648,301.0,11.501,raj mafia member one day meet girl meera chasi...


In [4]:
# Check for missing values and remove them
df_cleaned = df.dropna(subset=['cleaned_overview', 'language', 'genre'])
df_cleaned.head()

,title,release_year,language,genre,overview,vote_average,vote_count,popularity,cleaned_overview
0,Pad Man,2018,Indisch,Humor,upon realizing extent woman affected menses se...,7.420,200.0,7.036,upon realizing extent woman affected menses se...
1,Tamasha,2015,Indisch,Humor,meeting vacation ved tara sense connection vow...,6.720,141.0,8.770,meeting vacation ved tara sense connection vow...
2,Tu Jhoothi Main Makkaar,2023,Indisch,Humor,earn extra cash mickey help couple break life ...,6.253,144.0,10.045,earn extra cash mickey help couple break life ...
3,Hindi Medium,2017,Indisch,Humor,mita raj batra affluent couple delhi chandni c...,7.300,166.0,7.001,mita raj batra affluent couple delhi chandni c...
4,Dilwale,2015,Indisch,Humor,raj mafia member one day meet girl meera chasi...,6.648,301.0,11.501,raj mafia member one day meet girl meera chasi...


In [5]:
# Combine the 'cleaned_overview' and 'genre' columns properly to avoid the warning
df_cleaned = df_cleaned.copy()
df_cleaned.loc[:, 'combined_text'] = df_cleaned['cleaned_overview'].fillna('') + ' ' + df_cleaned['genre'].fillna('')


In [6]:
df_cleaned['cleaned_overview'] = df_cleaned['cleaned_overview'].fillna('')

In [7]:
# Define features (X) and target (y)
X = df_cleaned['combined_text']
y = df_cleaned['language']

In [8]:
from sklearn.preprocessing import LabelEncoder
# Convert the 'language' column to numeric labels using LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['language'])

In [9]:
# Create a new column 'label' to store the encoded language labels
df_cleaned['label'] = label_encoder.fit_transform(df_cleaned['language'])

In [10]:
# Create value tuples (label, cleaned_overview)
value_tuples = []
for _, row in df_cleaned.iterrows():
    value_tuples.append((row['label'], row['cleaned_overview']))

# Print the length and a few examples
print(f"Total number of value tuples: {len(value_tuples)}")
print("First tuple:", value_tuples[0])
print("Last tuple:", value_tuples[-1])

Total number of value tuples: 15499
First tuple: (3, 'upon realizing extent woman affected menses set create sanitary pad machine provide inexpensive sanitary pad woman rural india')
Last tuple: (4, 'several european underground director turn ancient rule mankind inside shocking goresoaked interpretation god law bible world ending commandment dying')


##### using a scikit-learn-based model

In [11]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### building a custom PyTorch classifier

In [12]:
from sklearn.model_selection import train_test_split

# Split the data into 80% training and 20% testing
#train_value_tuples, test_value_tuples = train_test_split(value_tuples, test_size=0.2, random_state=42)

# Print the size of the training and testing sets
#print(f"Training set size: {len(train_value_tuples)}")
#print(f"Testing set size: {len(test_value_tuples)}")


## Preparing data processing pipelines

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Vectorization using TF-IDF
vectorizer = TfidfVectorizer(max_features=30000)  # Limiting to top 5000 features we can use 30000 like knn to see the difference

#custom PyTorch

#X_train_tfidf = vectorizer.fit_transform([text for label, text in train_value_tuples])
#X_test_tfidf = vectorizer.transform([text for label, text in test_value_tuples])

# using a scikit-learn-based model

# Fit the vectorizer on the training data and transform it
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data using the fitted vectorizer
X_test_tfidf = vectorizer.transform(X_test)

In [14]:
import torch
# Converting TF-IDF matrices to dense tensors
X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_tfidf.toarray(), dtype=torch.float32)

### Create a Custom Dataset Class

In [15]:
from torch.utils.data import Dataset, DataLoader

class MovieDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

# Convert y_train and y_test to tensors
y_train_tensor = torch.tensor(label_encoder.transform(y_train.values), dtype=torch.long)
y_test_tensor = torch.tensor(label_encoder.transform(y_test.values), dtype=torch.long)

# Create Dataset objects
train_dataset = MovieDataset(X_train_tensor, y_train_tensor)
test_dataset = MovieDataset(X_test_tensor, y_test_tensor)

# Create DataLoader objects
BATCH_SIZE = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Define the PyTorch Model
define a simple feed-forward neural network model

In [16]:
#import torch.nn as nn
#import torch.nn.functional as F

#class TextClassificationModel(nn.Module):
#    def __init__(self, input_size, num_classes):
#        super(TextClassificationModel, self).__init__()
#        self.fc1 = nn.Linear(input_size, 128)
#        self.fc2 = nn.Linear(128, 64)
#        self.fc3 = nn.Linear(64, num_classes)
#
#    def forward(self, x):
#        x = F.relu(self.fc1(x))
#        x = F.relu(self.fc2(x))
#        x = self.fc3(x)
#        return x

#### Reduce Model Complexity
We will start by simplifying the model. A smaller network reduces the risk of overfitting, particularly when the dataset size is relatively small for a complex model

In [17]:
import torch.nn as nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(TextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Reduced the number of neurons
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [18]:
# Instantiate the model
input_size = X_train_tfidf.shape[1]
num_classes = len(label_encoder.classes_)
model = TextClassificationModel(input_size, num_classes)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


TextClassificationModel(
  (fc1): Linear(in_features=28191, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=6, bias=True)
)

## Define the Loss Function and Optimizer
We’ll use cross-entropy loss for this multi-class classification problem and an optimizer like Adam for optimization.

In [19]:
import torch.optim as optim

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Adding weight decay



## Training the Model
we feed batches of data to the model and optimize it using backpropagation

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        epoch_loss = total_loss / len(train_loader)
        epoch_acc = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

# Train the model for 10 epochs
train_model(model, train_loader, criterion, optimizer, num_epochs=10)


Epoch [1/10], Loss: 1.6232, Accuracy: 0.3138
Epoch [2/10], Loss: 1.0036, Accuracy: 0.6698
Epoch [3/10], Loss: 0.4040, Accuracy: 0.8889
Epoch [4/10], Loss: 0.1522, Accuracy: 0.9735
Epoch [5/10], Loss: 0.0588, Accuracy: 0.9930
Epoch [6/10], Loss: 0.0257, Accuracy: 0.9986
Epoch [7/10], Loss: 0.0134, Accuracy: 0.9994
Epoch [8/10], Loss: 0.0078, Accuracy: 0.9997
Epoch [9/10], Loss: 0.0049, Accuracy: 0.9998
Epoch [10/10], Loss: 0.0036, Accuracy: 0.9998


### Evaluate the Model
After training, we can evaluate our model’s performance on the test set.

In [21]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')

# Evaluate the model
evaluate_model(model, test_loader)


Test Accuracy: 0.7116


In [22]:
## without Reduce Model Complexity Test Accuracy: 0.7232
## with Reduce Model Complexity and L2 Regularization (Weight Decay) Test Accuracy: 0.6900
## with  Reduce Model Complexity Test Accuracy: 0.7300

In [23]:
import torch.optim as optim

# List of batch sizes to try
batch_sizes = [32, 64, 128, 256]

# Loop over each batch size
for batch_size in batch_sizes:
    print(f"Training with batch size: {batch_size}")
    
    # Create new DataLoader with the current batch size
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # Reinitialize the model
    model = TextClassificationModel(input_size, num_classes)
    model.to(device)

    # Reinitialize the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model for 10 epochs with the current batch size
    train_model(model, train_loader, criterion, optimizer, num_epochs=10)

    # Evaluate the model on the test set
    evaluate_model(model, test_loader)


Training with batch size: 32
Epoch [1/10], Loss: 1.5456, Accuracy: 0.3608
Epoch [2/10], Loss: 0.8460, Accuracy: 0.7174
Epoch [3/10], Loss: 0.3177, Accuracy: 0.9114
Epoch [4/10], Loss: 0.1028, Accuracy: 0.9777
Epoch [5/10], Loss: 0.0320, Accuracy: 0.9956
Epoch [6/10], Loss: 0.0114, Accuracy: 0.9992
Epoch [7/10], Loss: 0.0055, Accuracy: 0.9996
Epoch [8/10], Loss: 0.0033, Accuracy: 0.9998
Epoch [9/10], Loss: 0.0023, Accuracy: 0.9998
Epoch [10/10], Loss: 0.0019, Accuracy: 0.9998
Test Accuracy: 0.6874
Training with batch size: 64
Epoch [1/10], Loss: 1.6247, Accuracy: 0.3007
Epoch [2/10], Loss: 0.9652, Accuracy: 0.6676
Epoch [3/10], Loss: 0.4300, Accuracy: 0.8918
Epoch [4/10], Loss: 0.1741, Accuracy: 0.9664
Epoch [5/10], Loss: 0.0707, Accuracy: 0.9905
Epoch [6/10], Loss: 0.0313, Accuracy: 0.9977
Epoch [7/10], Loss: 0.0159, Accuracy: 0.9997
Epoch [8/10], Loss: 0.0094, Accuracy: 0.9998
Epoch [9/10], Loss: 0.0061, Accuracy: 0.9998
Epoch [10/10], Loss: 0.0043, Accuracy: 0.9998
Test Accuracy: 0.7

In [24]:
import torch.optim as optim

# Define different weight decay values to experiment with
weight_decay_values = [0, 1e-4, 1e-3, 1e-2]

# Loop over the different weight decay values
for weight_decay in weight_decay_values:
    print(f"Training with weight decay: {weight_decay}")
    
    # Reinitialize the model
    model = TextClassificationModel(input_size, num_classes)
    model.to(device)

    # Reinitialize the optimizer with the current weight decay
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=weight_decay)

    # Train the model for 10 epochs with the current weight decay
    train_model(model, train_loader, criterion, optimizer, num_epochs=10)

    # Evaluate the model on the test set
    evaluate_model(model, test_loader)


Training with weight decay: 0


Epoch [1/10], Loss: 1.7686, Accuracy: 0.2709
Epoch [2/10], Loss: 1.6026, Accuracy: 0.3297
Epoch [3/10], Loss: 1.2047, Accuracy: 0.6133
Epoch [4/10], Loss: 0.7728, Accuracy: 0.7882
Epoch [5/10], Loss: 0.4747, Accuracy: 0.8712
Epoch [6/10], Loss: 0.2890, Accuracy: 0.9389
Epoch [7/10], Loss: 0.1760, Accuracy: 0.9701
Epoch [8/10], Loss: 0.1102, Accuracy: 0.9869
Epoch [9/10], Loss: 0.0711, Accuracy: 0.9939
Epoch [10/10], Loss: 0.0475, Accuracy: 0.9968
Test Accuracy: 0.7032
Training with weight decay: 0.0001
Epoch [1/10], Loss: 1.7653, Accuracy: 0.2664
Epoch [2/10], Loss: 1.6113, Accuracy: 0.2947
Epoch [3/10], Loss: 1.3043, Accuracy: 0.5116
Epoch [4/10], Loss: 0.8724, Accuracy: 0.7830
Epoch [5/10], Loss: 0.5078, Accuracy: 0.9016
Epoch [6/10], Loss: 0.2939, Accuracy: 0.9512
Epoch [7/10], Loss: 0.1787, Accuracy: 0.9764
Epoch [8/10], Loss: 0.1143, Accuracy: 0.9889
Epoch [9/10], Loss: 0.0789, Accuracy: 0.9949
Epoch [10/10], Loss: 0.0580, Accuracy: 0.9982
Test Accuracy: 0.7126
Training with weigh

In [25]:
learning_rates = [1e-4, 1e-3, 1e-2]

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")
    
    # Reinitialize the model
    model = TextClassificationModel(input_size, num_classes)
    model.to(device)

    # Reinitialize the optimizer with the current learning rate
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model with the current learning rate
    train_model(model, train_loader, criterion, optimizer, num_epochs=10)

    # Evaluate the model on the test set
    evaluate_model(model, test_loader)


Training with learning rate: 0.0001


Epoch [1/10], Loss: 1.7930, Accuracy: 0.0974
Epoch [2/10], Loss: 1.7851, Accuracy: 0.0974
Epoch [3/10], Loss: 1.7740, Accuracy: 0.2045
Epoch [4/10], Loss: 1.7593, Accuracy: 0.3839
Epoch [5/10], Loss: 1.7412, Accuracy: 0.4589
Epoch [6/10], Loss: 1.7188, Accuracy: 0.4187
Epoch [7/10], Loss: 1.6909, Accuracy: 0.3507
Epoch [8/10], Loss: 1.6612, Accuracy: 0.3057
Epoch [9/10], Loss: 1.6297, Accuracy: 0.2988
Epoch [10/10], Loss: 1.5954, Accuracy: 0.3058
Test Accuracy: 0.3039
Training with learning rate: 0.001
Epoch [1/10], Loss: 1.7511, Accuracy: 0.2274
Epoch [2/10], Loss: 1.5941, Accuracy: 0.3021
Epoch [3/10], Loss: 1.2156, Accuracy: 0.6271
Epoch [4/10], Loss: 0.7059, Accuracy: 0.8623
Epoch [5/10], Loss: 0.3587, Accuracy: 0.9375
Epoch [6/10], Loss: 0.1907, Accuracy: 0.9723
Epoch [7/10], Loss: 0.1084, Accuracy: 0.9892
Epoch [8/10], Loss: 0.0657, Accuracy: 0.9958
Epoch [9/10], Loss: 0.0425, Accuracy: 0.9985
Epoch [10/10], Loss: 0.0288, Accuracy: 0.9993
Test Accuracy: 0.7119
Training with learn